In [116]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor
from llm import LLMInteractor
from runner import process_threads, check_and_post_reminders

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
slack_interactor = SlackInteractor()
llm_interactor = LLMInteractor()

In [162]:
print("\nFetching new messages...")
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(slack_interactor, llm_interactor, threads)

print("\nChecking for due items and posting reminders...")
check_and_post_reminders(slack_interactor, llm_interactor)


Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 87 old messages
Found 0 new messages
Updated complete_conversations.pkl with new messages
Found 0 threads with new messages.

Checking for due items and posting reminders...
